In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/studentcnn/pytorch/default/1/model (4).pth


In [2]:
!pip install detectors -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.8/616.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.7 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader


In [4]:
import torch,timm,detectors
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import KLDivLoss
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128 , shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = timm.create_model("resnet34_cifar10", pretrained=True).to(device)


100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s]
Downloading: "https://huggingface.co/edadaltocg/resnet34_cifar10/resolve/main/pytorch_model.bin" to /root/.cache/torch/hub/checkpoints/resnet34_cifar10.pth
100%|██████████| 81.3M/81.3M [00:00<00:00, 281MB/s]


In [5]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split into validation and test sets (50/50)
val_size = len(cifar10_test) // 2
test_size = len(cifar10_test) - val_size
val_set, final_test_set = random_split(cifar10_test, [val_size, test_size])

val_loader = DataLoader(val_set, batch_size=128, shuffle=False)
test_loader = DataLoader(final_test_set, batch_size=128, shuffle=False)


In [6]:
# Define the StudentCNN model
class StudentCNN(nn.Module):
    def __init__(self):
        super(StudentCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # Adjusted for CIFAR-10
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 -> 16x16
        x = self.pool(F.relu(self.conv2(x)))  # 16x16 -> 8x8
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [7]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [8]:
def train_with_kl_and_ce(student_model, teacher_model, loader, optimizer, epochs=600, temperature=2, alpha=0.5):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    ce_loss = nn.CrossEntropyLoss()
    student_model.train()
    teacher_model.eval()
    best_val_acc = 0
    patience = 15
    epochs_no_improve = 0
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher_model(images)
                teacher_probs = F.softmax(teacher_logits / temperature, dim=1)
              
            optimizer.zero_grad()
            student_logits = student_model(images)

            # KL Divergence Loss
            student_log_probs = F.log_softmax(student_logits / temperature, dim=1)

            loss_kl = kl_loss(student_log_probs, teacher_probs) * (temperature ** 2)

            # Cross-Entropy Loss
            loss_ce = ce_loss(student_logits, labels)

            # Combined Loss
            loss = alpha * loss_kl + (1 - alpha) * loss_ce

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        val_acc = evaluate(student_model, val_loader)
        
        if val_acc > best_val_acc:
                best_val_acc = val_acc
                epochs_no_improve = 0
                # Save best model
                torch.save(student_model.state_dict(), "best_student_model.pth")
        else:
                epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        print(f"Epoch {epoch+1}, Total Loss: {total_loss:.4f} Validation :{val_acc}")


In [9]:
student_model_kl = StudentCNN()

In [10]:


student_model_kl.to(device)
optimizer_kl =torch.optim.Adam(student_model_kl.parameters(), lr=1e-3, weight_decay=5e-4)



train_with_kl_and_ce(student_model_kl, teacher_model, train_loader, optimizer_kl)

Epoch 1, Total Loss: 1211.9782 Validation :62.42
Epoch 2, Total Loss: 863.0903 Validation :68.42
Epoch 3, Total Loss: 722.2231 Validation :70.54
Epoch 4, Total Loss: 606.2301 Validation :72.06
Epoch 5, Total Loss: 510.0930 Validation :73.1
Epoch 6, Total Loss: 414.6368 Validation :74.98
Epoch 7, Total Loss: 337.4797 Validation :74.16
Epoch 8, Total Loss: 261.2125 Validation :74.28
Epoch 9, Total Loss: 205.5336 Validation :73.72
Epoch 10, Total Loss: 165.2303 Validation :74.94
Epoch 11, Total Loss: 138.6143 Validation :74.06
Epoch 12, Total Loss: 126.1792 Validation :73.16
Epoch 13, Total Loss: 135.3498 Validation :72.76
Epoch 14, Total Loss: 125.6618 Validation :72.52
Epoch 15, Total Loss: 99.5081 Validation :72.96
Epoch 16, Total Loss: 104.4281 Validation :71.22
Epoch 17, Total Loss: 120.7899 Validation :73.36
Epoch 18, Total Loss: 122.6027 Validation :72.7
Epoch 19, Total Loss: 90.2643 Validation :73.22
Epoch 20, Total Loss: 72.5394 Validation :73.6
Early stopping at epoch 21


In [11]:
torch.save(student_model_kl, '/kaggle/working/model.pth')

In [12]:
correct = 0
total = 0
# test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=128 , shuffle=False)
with torch.no_grad():
    for images, labels in test_loader:
        outputs = student_model_kl(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 72.90%
